# Branch-and-cut for the 0–1 Knapsack Problem

## Task 1: Primal Heuristic

In [ ]:
#testing variables:

p=[5,4,3,5,4,5]
w=[2,2,2,4,4,6,]

items = [
    {"w": 2, "p": 5},
    {"w": 2, "p": 4},
    {"w": 2, "p": 3},
    {"w": 4, "p": 5}
]

c = 10
a=c

In [16]:
import random

def generate_random_items(num_items):
    items = []
    for _ in range(num_items):
        # Generate random weights and profits within a similar range as the example
        weight = random.randint(1, 10)  # Random weight between 1 and 10
        profit = random.randint(1, 10)  # Random profit between 1 and 10
        items.append({"w": weight, "p": profit})
    return items

# Generate a list of 1000 items
random_items = generate_random_items(1000)

# Display the first 10 items to check
for item in random_items[:10]:
    print(item)


{'w': 5, 'p': 10}
{'w': 8, 'p': 10}
{'w': 7, 'p': 8}
{'w': 5, 'p': 4}
{'w': 8, 'p': 8}
{'w': 2, 'p': 4}
{'w': 7, 'p': 6}
{'w': 1, 'p': 10}
{'w': 10, 'p': 3}
{'w': 6, 'p': 10}


In [18]:
def knapsack_greedy_heuristic(items, c):
    #no need to sort
    items=sorted(items, key = lambda x: x['p']/x['w'], reverse = True)
    total_value=0
    total_weight=0
    packed_items = []
    for item in items:
        if total_weight + item['w']<=c:
            packed_items.append(item)
            total_weight += item['w']
            total_value += item ['p']
    return total_value, packed_items, total_weight

print(knapsack_greedy_heuristic(random_items, c))


(99, [{'w': 1, 'p': 10}, {'w': 1, 'p': 10}, {'w': 1, 'p': 10}, {'w': 1, 'p': 10}, {'w': 1, 'p': 10}, {'w': 1, 'p': 10}, {'w': 1, 'p': 10}, {'w': 1, 'p': 10}, {'w': 1, 'p': 10}, {'w': 1, 'p': 9}], 10)


Lets create our shortest path code

# OUR CODE 

In [ ]:
Vertex = int #(0,1,2,3,4,...)
Arc = Tuple[Vertex, Vertex]
Tour = List[Vertex]

class TSPIntance:
    n: int
    x: List[float]
    y: List[float]
    cost: Dict[Arc, float]

    def __init__(self, x: List[float], y: List[float]):
        assert len(x) == len(y), "X and Y coordinate lists must have the same length"

        self.n = len(x)
        self.x = x
        self.y = y
        self.cost = {
            (i, j): ((self.x[i] - self.x[j])**2 + (self.y[i] - self.y[j])**2)**0.5
            for i in self.vertices()
            for j in self.vertices()
            if i != j
        }

    def vertices(self) -> Iterable[Vertex]:
        return range(self.n)
    
    def arcs(self) -> KeysView:
        return self.cost.keys()

    @staticmethod #create a TSP instance to test our code
    def random(n: int) -> TSPIntance:
        x = [uniform(0, 10) for _ in range(n)]
        y = [uniform(0, 10) for _ in range(n)]
        return TSPIntance(x=x, y=y)

In [ ]:
class TSPSolution:
    tour: Tour
    cost: float

    def __init__(self, tour: Tour, **kwargs):
        assert 'cost' in kwargs or 'instance' in kwargs, \
            "You must pass the tour cost or a TSP instance to compute it"

        if 'cost' in kwargs:
            self.cost = kwargs.get('cost')
        elif 'instance' in kwargs:
            tsp = kwargs.get('instance')
            self.cost = sum(
                tsp.cost[i, j]
                for i in tour[:-1]
                for j in tour[1:]
            )

        self.tour = tour

    def __str__(self) -> str:
        return "[" + ', '.join(map(str, self.tour)) + f"] - Cost: {self.cost:.2f}"

In [ ]:
class BranchAndCutIntegerSolver:
    tsp: TSPIntance
    m: Model
    x: tupledict

    def __init__(self, tsp: TSPIntance):
        self.tsp = tsp
        self.m = Model()
        self.x = self.m.addVars(self.tsp.arcs(), obj=self.tsp.cost, vtype=GRB.BINARY, name='x')
        self.__build_model()

    def __build_model(self) -> None:
        self.m.addConstrs(self.x.sum(i, '*') == 1 for i in self.tsp.vertices())#sums ij: we add a constraint for all i's #constraint: all only one incoming trip to a customer
        self.m.addConstrs(self.x.sum('*', i) == 1 for i in self.tsp.vertices())#sums ji: we add a constraint #only one outgoing trip to customer

    def solve(self) -> TSPSolution:
        self.m.setParam(GRB.Param.LazyConstraints, 1) #lazy sonstraints are going to be used
        self.m.optimize(lambda _, where: self.__separate(where=where)) #optimize: solve this gruobi! we sepcify 2 parameters to check finding the subgroups 
        #lamda has two arguments: The model and the int number: _ (=ignoring the Model as we have 
        # it safed anyways in the class BranchAndCutIntegerSolver "model") and 

        if self.m.Status != GRB.OPTIMAL:
            raise RuntimeError("Could not solve TSP model to optimality")
        
        return TSPSolution(tour=self.__tour_starting_at(0), cost=self.m.ObjVal)
    
    def __separate(self, where: int) -> None: #= __find_subtours
        if where != GRB.Callback.MIPSOL: #See documentations of callback online, SO: when we don't find a feasable int solution we return
            return
        
        remaining = set(self.tsp.vertices()) #the nodes we still have to explore, at first we have all of them

        while len(remaining) > 0:
            # Get the first vertex of the set
            start = remaining.pop()
            #start = next(iter(remaining))
            subtour = self.__tour_starting_at(start)

            if len(subtour) == self.tsp.n:
                # Feasible tour visiting all vertices
                return
            
            self.__add_sec_for(subtour)

            remaining -= set(subtour) #convert subtour into set because we cannot do set-= list

    def __tour_starting_at(self, i: Vertex) -> Tour:#e.g.: subset (2,3,7), i=2, tour =[2], current = 3 -next iteration: tour = [2,3], current = 7, -next iteration: tour = [2,3,7], current = 2 -
        #2 is not different form 2 so the last one is the subtour
        tour = [i]
        current = self.__next_vertex(i=i)

        while current != i:
            tour.append(current)
            current = self.__next_vertex(current)

        return tour

    def __next_vertex(self, i: Vertex) -> Vertex: #check which vertex we are going to 
        for j in self.tsp.vertices():#for all vertices except itself, check which one we are going to:
            if j == i:
                continue

            try:
                # When in a callback
                x = self.m.cbGetSolution(self.x[i,j]) #we cannot use x[i,j].X when using callbacki
            except GurobiError: #if we fail we use x[i,j].X
                # When optimisation is over
                x = self.x[i,j].X #2 is not different form 2 s

            if x > 0.5:
                return j
            
        raise RuntimeError(f"Ve2 is not different form 2 srtex {i} has no successor!")
    
#m.cbGetSolution(x[i,j]) during a callback
#x[i,j].X after optimization
    def __add_sec_for(self, subtour: Tour) -> None:
        print("Adding a SEC/subtour for [" + ', '.join(map(str, subtour)) + "]")
        self.m.cbLazy(
            sum(
                self.x[i, j]
                for i, j in self.tsp.arcs() #using arcs function that contains all valid arcs
                if i in subtour and j not in subtour
            ) >= 1
        )

In [ ]:
tsp = TSPIntance.random(n=20)
solver = BranchAndCutIntegerSolver(tsp=tsp)
solution = solver.solve()